In [1]:
import pandas as pd
import re
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from bs4 import BeautifulSoup
import re
import pandas as pd
#from tabulate import tabulate
import os
#import chromedriver_binary
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import datetime as dt
from dateutil.parser import parse

In [2]:
AS= pd.read_csv('../data/American_Airlines.csv',skiprows=7)


In [3]:
df= pd.DataFrame(AS)
print(len(df))

80796


In [4]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
25,AA,01/01/2015,290.0,N3JKAA,AUS,14:35,00:00
111,AA,01/01/2017,106.0,N3HYAA,PHX,16:53,00:00
229,AA,01/01/2019,2514.0,N882NN,AUS,17:21,00:00
439,AA,01/02/2019,10.0,N103NN,LAX,05:17,00:00
457,AA,01/02/2019,310.0,N349AN,DFW,16:31,00:00
...,...,...,...,...,...,...,...
80076,AA,12/28/2015,1638.0,N3KHAA,CLT,21:59,00:00
80230,AA,12/28/2018,2651.0,N841NN,PHX,07:05,00:00
80249,AA,12/29/2015,94.0,N3EKAA,SAN,16:04,00:00
80272,AA,12/29/2015,1312.0,N3GDAA,PHX,20:15,00:00


In [5]:
df= df.dropna(subset=['Scheduled Arrival Time','Actual Arrival Time'])
df.head()

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,AA,01/01/2015,2.0,N795AA,LAX,17:35,16:51
1,AA,01/01/2015,4.0,N799AA,LAX,20:50,20:33
2,AA,01/01/2015,10.0,N796AA,LAX,05:59,05:44
3,AA,01/01/2015,12.0,N792AA,LAX,19:50,19:35
4,AA,01/01/2015,16.0,N783AA,SFO,21:15,21:49


In [6]:
#Rename it to scheduled arrival date
df.rename(columns = {'Date (MM/DD/YYYY)':'Scheduled Date'}, inplace = True) 


In [7]:
df.head()

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,AA,01/01/2015,2.0,N795AA,LAX,17:35,16:51
1,AA,01/01/2015,4.0,N799AA,LAX,20:50,20:33
2,AA,01/01/2015,10.0,N796AA,LAX,05:59,05:44
3,AA,01/01/2015,12.0,N792AA,LAX,19:50,19:35
4,AA,01/01/2015,16.0,N783AA,SFO,21:15,21:49


In [8]:
df['Scheduled Date']= pd.to_datetime(df['Scheduled Date'])
type(df['Scheduled Date'][0])
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,AA,2015-01-01,2.0,N795AA,LAX,17:35,16:51
1,AA,2015-01-01,4.0,N799AA,LAX,20:50,20:33
2,AA,2015-01-01,10.0,N796AA,LAX,05:59,05:44
3,AA,2015-01-01,12.0,N792AA,LAX,19:50,19:35
4,AA,2015-01-01,16.0,N783AA,SFO,21:15,21:49
...,...,...,...,...,...,...,...
80790,AA,2018-12-31,2572.0,N389AA,MIA,21:29,21:50
80791,AA,2018-12-31,2651.0,N973NN,PHX,07:05,07:04
80792,AA,2018-12-31,2654.0,N912NN,DCA,19:50,19:19
80793,AA,2018-12-31,2731.0,N987NN,CLT,14:58,15:33


In [9]:
# Makea new column called actual arrival date 
df['Actual Arrival Date'] = df['Scheduled Date']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
0,AA,2015-01-01,2.0,N795AA,LAX,17:35,16:51,2015-01-01
1,AA,2015-01-01,4.0,N799AA,LAX,20:50,20:33,2015-01-01
2,AA,2015-01-01,10.0,N796AA,LAX,05:59,05:44,2015-01-01
3,AA,2015-01-01,12.0,N792AA,LAX,19:50,19:35,2015-01-01
4,AA,2015-01-01,16.0,N783AA,SFO,21:15,21:49,2015-01-01
...,...,...,...,...,...,...,...,...
80790,AA,2018-12-31,2572.0,N389AA,MIA,21:29,21:50,2018-12-31
80791,AA,2018-12-31,2651.0,N973NN,PHX,07:05,07:04,2018-12-31
80792,AA,2018-12-31,2654.0,N912NN,DCA,19:50,19:19,2018-12-31
80793,AA,2018-12-31,2731.0,N987NN,CLT,14:58,15:33,2018-12-31


In [10]:
df.loc[df['Actual Arrival Time']=='24:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
286,AA,2016-01-02,22.0,N796AA,LAX,23:32,24:00,2016-01-02
1302,AA,2016-01-06,1898.0,N509AY,CLT,23:56,24:00,2016-01-06
3054,AA,2019-01-13,166.0,N113AN,SFO,00:08,24:00,2019-01-13
10484,AA,2018-02-14,1608.0,N924US,CLT,23:57,24:00,2018-02-14
12317,AA,2016-02-22,1898.0,N170US,CLT,23:56,24:00,2016-02-22
...,...,...,...,...,...,...,...,...
69923,AA,2015-11-03,1898.0,N584UW,CLT,23:59,24:00,2015-11-03
76814,AA,2015-12-11,1898.0,N152UW,CLT,23:59,24:00,2015-12-11
76999,AA,2015-12-12,1898.0,N586UW,CLT,23:59,24:00,2015-12-12
77882,AA,2018-12-16,2201.0,N107NN,LAX,23:48,24:00,2018-12-16


In [11]:
df['Actual Arrival Time']= df['Actual Arrival Time'].str.replace('24:00','00:00')

In [12]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
25,AA,2015-01-01,290.0,N3JKAA,AUS,14:35,00:00,2015-01-01
111,AA,2017-01-01,106.0,N3HYAA,PHX,16:53,00:00,2017-01-01
229,AA,2019-01-01,2514.0,N882NN,AUS,17:21,00:00,2019-01-01
286,AA,2016-01-02,22.0,N796AA,LAX,23:32,00:00,2016-01-02
439,AA,2019-01-02,10.0,N103NN,LAX,05:17,00:00,2019-01-02
...,...,...,...,...,...,...,...,...
80076,AA,2015-12-28,1638.0,N3KHAA,CLT,21:59,00:00,2015-12-28
80230,AA,2018-12-28,2651.0,N841NN,PHX,07:05,00:00,2018-12-28
80249,AA,2015-12-29,94.0,N3EKAA,SAN,16:04,00:00,2015-12-29
80272,AA,2015-12-29,1312.0,N3GDAA,PHX,20:15,00:00,2015-12-29


In [13]:
df.loc[df['Actual Arrival Time']=='00:00','Actual Arrival Date']+=dt.timedelta(days=1)

In [14]:
df['Actual Arrival Date']= df['Actual Arrival Date'].dt.strftime('%Y-%m-%d')

In [15]:
df['Scheduled Date']= df['Scheduled Date'].dt.strftime('%Y-%m-%d')

In [16]:
df['Actual Arrival']=df['Actual Arrival Date'] + ' ' + df['Actual Arrival Time']

In [17]:
df['Scheduled Arrival']=df['Scheduled Date'] + ' ' + df['Scheduled Arrival Time']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,AA,2015-01-01,2.0,N795AA,LAX,17:35,16:51,2015-01-01,2015-01-01 16:51,2015-01-01 17:35
1,AA,2015-01-01,4.0,N799AA,LAX,20:50,20:33,2015-01-01,2015-01-01 20:33,2015-01-01 20:50
2,AA,2015-01-01,10.0,N796AA,LAX,05:59,05:44,2015-01-01,2015-01-01 05:44,2015-01-01 05:59
3,AA,2015-01-01,12.0,N792AA,LAX,19:50,19:35,2015-01-01,2015-01-01 19:35,2015-01-01 19:50
4,AA,2015-01-01,16.0,N783AA,SFO,21:15,21:49,2015-01-01,2015-01-01 21:49,2015-01-01 21:15
...,...,...,...,...,...,...,...,...,...,...
80790,AA,2018-12-31,2572.0,N389AA,MIA,21:29,21:50,2018-12-31,2018-12-31 21:50,2018-12-31 21:29
80791,AA,2018-12-31,2651.0,N973NN,PHX,07:05,07:04,2018-12-31,2018-12-31 07:04,2018-12-31 07:05
80792,AA,2018-12-31,2654.0,N912NN,DCA,19:50,19:19,2018-12-31,2018-12-31 19:19,2018-12-31 19:50
80793,AA,2018-12-31,2731.0,N987NN,CLT,14:58,15:33,2018-12-31,2018-12-31 15:33,2018-12-31 14:58


In [18]:
FMT = '%Y-%m-%d %H:%M'
df['Scheduled Arrival']= pd.to_datetime(df['Scheduled Arrival'], format=FMT)
df['Actual Arrival']= pd.to_datetime(df['Actual Arrival'], format=FMT)
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,AA,2015-01-01,2.0,N795AA,LAX,17:35,16:51,2015-01-01,2015-01-01 16:51:00,2015-01-01 17:35:00
1,AA,2015-01-01,4.0,N799AA,LAX,20:50,20:33,2015-01-01,2015-01-01 20:33:00,2015-01-01 20:50:00
2,AA,2015-01-01,10.0,N796AA,LAX,05:59,05:44,2015-01-01,2015-01-01 05:44:00,2015-01-01 05:59:00
3,AA,2015-01-01,12.0,N792AA,LAX,19:50,19:35,2015-01-01,2015-01-01 19:35:00,2015-01-01 19:50:00
4,AA,2015-01-01,16.0,N783AA,SFO,21:15,21:49,2015-01-01,2015-01-01 21:49:00,2015-01-01 21:15:00
...,...,...,...,...,...,...,...,...,...,...
80790,AA,2018-12-31,2572.0,N389AA,MIA,21:29,21:50,2018-12-31,2018-12-31 21:50:00,2018-12-31 21:29:00
80791,AA,2018-12-31,2651.0,N973NN,PHX,07:05,07:04,2018-12-31,2018-12-31 07:04:00,2018-12-31 07:05:00
80792,AA,2018-12-31,2654.0,N912NN,DCA,19:50,19:19,2018-12-31,2018-12-31 19:19:00,2018-12-31 19:50:00
80793,AA,2018-12-31,2731.0,N987NN,CLT,14:58,15:33,2018-12-31,2018-12-31 15:33:00,2018-12-31 14:58:00


In [19]:
df['Difference']= (df['Scheduled Arrival']-df['Actual Arrival']).astype('timedelta64[m]')
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival,Difference
0,AA,2015-01-01,2.0,N795AA,LAX,17:35,16:51,2015-01-01,2015-01-01 16:51:00,2015-01-01 17:35:00,44.0
1,AA,2015-01-01,4.0,N799AA,LAX,20:50,20:33,2015-01-01,2015-01-01 20:33:00,2015-01-01 20:50:00,17.0
2,AA,2015-01-01,10.0,N796AA,LAX,05:59,05:44,2015-01-01,2015-01-01 05:44:00,2015-01-01 05:59:00,15.0
3,AA,2015-01-01,12.0,N792AA,LAX,19:50,19:35,2015-01-01,2015-01-01 19:35:00,2015-01-01 19:50:00,15.0
4,AA,2015-01-01,16.0,N783AA,SFO,21:15,21:49,2015-01-01,2015-01-01 21:49:00,2015-01-01 21:15:00,-34.0
...,...,...,...,...,...,...,...,...,...,...,...
80790,AA,2018-12-31,2572.0,N389AA,MIA,21:29,21:50,2018-12-31,2018-12-31 21:50:00,2018-12-31 21:29:00,-21.0
80791,AA,2018-12-31,2651.0,N973NN,PHX,07:05,07:04,2018-12-31,2018-12-31 07:04:00,2018-12-31 07:05:00,1.0
80792,AA,2018-12-31,2654.0,N912NN,DCA,19:50,19:19,2018-12-31,2018-12-31 19:19:00,2018-12-31 19:50:00,31.0
80793,AA,2018-12-31,2731.0,N987NN,CLT,14:58,15:33,2018-12-31,2018-12-31 15:33:00,2018-12-31 14:58:00,-35.0


In [20]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2019
Count_2019=df[df['Actual Arrival Date'].str.contains("2019")]
df19= pd.DataFrame(Count_2019)
Total_19=len(df19)
Twenty_Mins_Early_19= len(df19[df19['Difference'] > 20.0])
Early_Arrivals_19= len(df19[df19['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2019:",Total_19)
print("Flights that arrived early in 2019:",Early_Arrivals_19)
print("Flights that arrived more than 20 mins early in 2019:",Twenty_Mins_Early_19)

Total Alaska Airlines flights that landed at JFK in 2019: 10571
Flights that arrived early in 2019: 6669
Flights that arrived more than 20 mins early in 2019: 2529


In [22]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2018
Count_2018=df[df['Actual Arrival Date'].str.contains("2018")]
df18= pd.DataFrame(Count_2018)
Total_18=len(df18)
Twenty_Mins_Early_18= len(df18[df18['Difference'] > 20.0])
Early_Arrivals_18= len(df18[df18['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2018:",Total_18)
print("Flights that arrived early in 2018:",Early_Arrivals_18)
print("Flights that arrived more than 20 mins early in 2018:",Twenty_Mins_Early_18)

Total Alaska Airlines flights that landed at JFK in 2018: 17445
Flights that arrived early in 2018: 10929
Flights that arrived more than 20 mins early in 2018: 4320


In [23]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2017
Count_2017=df[df['Actual Arrival Date'].str.contains("2017")]
df17= pd.DataFrame(Count_2017)
Total_17=len(df17)
Twenty_Mins_Early_17= len(df17[df17['Difference'] > 20.0])
Early_Arrivals_17= len(df17[df17['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2017:",Total_17)
print("Flights that arrived early in 2017:",Early_Arrivals_17)
print("Flights that arrived more than 20 mins early in 2017:",Twenty_Mins_Early_17)

Total Alaska Airlines flights that landed at JFK in 2017: 17023
Flights that arrived early in 2017: 10180
Flights that arrived more than 20 mins early in 2017: 3923


In [24]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2016
Count_2016=df[df['Actual Arrival Date'].str.contains("2016")]
df16= pd.DataFrame(Count_2016)
Total_16=len(df16)
Twenty_Mins_Early_16= len(df16[df16['Difference'] > 20.0])
Early_Arrivals_16= len(df16[df16['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2016:",Total_16)
print("Flights that arrived early in 2016:",Early_Arrivals_16)
print("Flights that arrived more than 20 mins early in 2016:",Twenty_Mins_Early_16)

Total Alaska Airlines flights that landed at JFK in 2016: 18649
Flights that arrived early in 2016: 10876
Flights that arrived more than 20 mins early in 2016: 3479


In [25]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2015
Count_2015=df[df['Actual Arrival Date'].str.contains("2015")]
df15= pd.DataFrame(Count_2015)
Total_15=len(df15)
Twenty_Mins_Early_15= len(df15[df15['Difference'] > 20.0])
Early_Arrivals_15= len(df15[df15['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2015:",Total_15)
print("Flights that arrived early in 2015:",Early_Arrivals_15)
print("Flights that arrived more than 20 mins early in 2015:",Twenty_Mins_Early_15)

Total Alaska Airlines flights that landed at JFK in 2015: 17107
Flights that arrived early in 2015: 10761
Flights that arrived more than 20 mins early in 2015: 4074


In [ ]:
#df.loc[df['Actual Arrival Time']=='00:00']

In [27]:
#Final Results 
Count_2019=df[df['Actual Arrival Date'].str.contains("2019")]
df19= pd.DataFrame(Count_2019)
Total_19=len(df19)
Twenty_Mins_Early_19= len(df19[df19['Difference'] > 20.0])
Early_Arrivals_19= len(df19[df19['Difference'] > 0])
print("Total American Airlines flights that landed at JFK in 2019:",Total_19)
print("Flights that arrived early in 2019:",Early_Arrivals_19)
print("Flights that arrived more than 20 mins early in 2019:",Twenty_Mins_Early_19)
Count_2018=df[df['Actual Arrival Date'].str.contains("2018")]
df18= pd.DataFrame(Count_2018)
Total_18=len(df18)
Twenty_Mins_Early_18= len(df18[df18['Difference'] > 20.0])
Early_Arrivals_18= len(df18[df18['Difference'] > 0])
print("Total American Airlines flights that landed at JFK in 2018:",Total_18)
print("Flights that arrived early in 2018:",Early_Arrivals_18)
print("Flights that arrived more than 20 mins early in 2018:",Twenty_Mins_Early_18)
Count_2017=df[df['Actual Arrival Date'].str.contains("2017")]
df17= pd.DataFrame(Count_2017)
Total_17=len(df17)
Twenty_Mins_Early_17= len(df17[df17['Difference'] > 20.0])
Early_Arrivals_17= len(df17[df17['Difference'] > 0])
print("Total American Airlines flights that landed at JFK in 2017:",Total_17)
print("Flights that arrived early in 2017:",Early_Arrivals_17)
print("Flights that arrived more than 20 mins early in 2017:",Twenty_Mins_Early_17)
Count_2016=df[df['Actual Arrival Date'].str.contains("2016")]
df16= pd.DataFrame(Count_2016)
Total_16=len(df16)
Twenty_Mins_Early_16= len(df16[df16['Difference'] > 20.0])
Early_Arrivals_16= len(df16[df16['Difference'] > 0])
print("Total American Airlines flights that landed at JFK in 2016:",Total_16)
print("Flights that arrived early in 2016:",Early_Arrivals_16)
print("Flights that arrived more than 20 mins early in 2016:",Twenty_Mins_Early_16)
Count_2015=df[df['Actual Arrival Date'].str.contains("2015")]
df15= pd.DataFrame(Count_2015)
Total_15=len(df15)
Twenty_Mins_Early_15= len(df15[df15['Difference'] > 20.0])
Early_Arrivals_15= len(df15[df15['Difference'] > 0])
print("Total American Airlines flights that landed at JFK in 2015:",Total_15)
print("Flights that arrived early in 2015:",Early_Arrivals_15)
print("Flights that arrived more than 20 mins early in 2015:",Twenty_Mins_Early_15)

# dictionary of lists  
 
year =["2015","2016","2017","2018","2019"]
total_flights= [Total_15, Total_16, Total_17, Total_18, Total_19]
early_arrival=[Early_Arrivals_15,Early_Arrivals_16,Early_Arrivals_17,Early_Arrivals_18,Early_Arrivals_19]
inflated_padding=[Twenty_Mins_Early_15,Twenty_Mins_Early_16,Twenty_Mins_Early_17,Twenty_Mins_Early_18,Twenty_Mins_Early_19]

dict = {'year': year, 'Total flights': total_flights, 'Early Arrivals': early_arrival, 'Inflated padding(>20mins early)': inflated_padding} 
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv(r'../output/American_Airlines.csv',index=False) 

Total American Airlines flights that landed at JFK in 2019: 10571
Flights that arrived early in 2019: 6669
Flights that arrived more than 20 mins early in 2019: 2529
Total American Airlines flights that landed at JFK in 2018: 17445
Flights that arrived early in 2018: 10929
Flights that arrived more than 20 mins early in 2018: 4320
Total American Airlines flights that landed at JFK in 2017: 17023
Flights that arrived early in 2017: 10180
Flights that arrived more than 20 mins early in 2017: 3923
Total American Airlines flights that landed at JFK in 2016: 18649
Flights that arrived early in 2016: 10876
Flights that arrived more than 20 mins early in 2016: 3479
Total American Airlines flights that landed at JFK in 2015: 17107
Flights that arrived early in 2015: 10761
Flights that arrived more than 20 mins early in 2015: 4074
